<a href="https://colab.research.google.com/github/AndyZhang127/temp/blob/main/Chipotle_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Excercise3.Chipotle_pyspark







1.   read url: SparkFiles.get();addFile()
2.   def a function-->lambda--->udf()--->withColumn()
3.   types
4.   functions
5.   printSchema()
6.   select()
7.   orderBy()
8.   groupBy()
9.   filter()
10.  where()
11.  distinct()
12.  count()



In [ ]:
# Step 1. Import the necessary libraries

import pandas as pd
from pyspark.sql  import SparkSession
from pyspark  import SparkFiles

spark=SparkSession.builder.appName('Chipotle').getOrCreate()
spark

In [ ]:
# Step 2. Import the dataset from this address.
url="https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
spark.sparkContext.addFile(url)

In [ ]:
# Step 3. Assign it to a variable called chipo.
chipo=spark.read.csv(SparkFiles.get('chipotle.tsv'),sep='\t',header=True,inferSchema=True)
chipo.show()

In [ ]:
# Step 4. How many products cost more than $10.00?
from pyspark.sql.functions import udf,col
from pyspark.sql.types import FloatType,StringType

# processing item_price, cast string into float : def a function-->lambda--->udf()--->withColumn()
def dolariser(x):
  return str(x[1:-1])

udf_dolariser=udf(lambda x:dolariser(x),StringType())

chipo_df = chipo.withColumn("item_price",udf_dolariser(col("item_price")))

chipo_df.printSchema()

chipo_df = chipo_df.withColumn("item_price",chipo_df["item_price"].cast('float'))

chipo_df.printSchema()

chipo_df.filter(col('item_price')>10).select('item_name').distinct().count()

In [ ]:
# Step 5. What is the price of each item?
chipo.select('item_name','item_price').show()

In [ ]:
# Step 6. Sort by the name of the item
chipo.select('item_name','item_price').orderBy('item_name').show()

In [ ]:
# Step 7. What was the quantity of the most expensive item ordered?

#chipo_df.groupBy("item_name").agg({"item_price":"sum","quantity":"sum"}).orderBy("sum(item_price)",ascending=0).select("sum(quantity)").head(1)[0][0]

chipo_df.orderBy(chipo_df["item_price"], ascending=0).select("item_name","quantity").show(1)

In [ ]:
#Step 8. How many times was a Veggie Salad Bowl ordered?

chipo.groupBy("item_name").count().filter(chipo.item_name=='Veggie Salad Bowl').show()

In [ ]:
#Step 9. How many times did someone order more than one Canned Soda?

chipo_df.where((chipo_df.item_name=="Canned Soda") & (chipo_df.quantity>1)).count()